In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q torch==1.4.0 
!pip install transformers==3.5.1 --quiet

In [3]:
# from transformers import RobertaTokenizer, RobertaForMaskedLM

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [4]:
!pip install datasets
from datasets import load_dataset

In [5]:
# import random,shutil,os
# pathtofiles = os.listdir('/content/drive/MyDrive/webdata_text/')
# randomlist = random.sample(range(0, 2096), 400)
# for i in randomlist:
#   shutil.copy(os.path.join('/content/drive/MyDrive/webdata_text/',pathtofiles[i]),'/content/drive/MyDrive/webdata_small/')

In [6]:
# len(pathtofiles)

In [7]:
import os
pathtofiles = os.listdir('/content/drive/MyDrive/webdata_small/')
string = '/content/drive/MyDrive/webdata_small/'
pathtofiles = list(map(lambda orig_string: string+orig_string,pathtofiles))

In [8]:
dataset = load_dataset('text',data_files=pathtofiles)

Using custom data configuration default-be2a9e38e4110afe


  0%|          | 0/417 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-be2a9e38e4110afe/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 829534
    })
})


In [10]:
model_checkpoint = 'roberta-base'
from transformers import AutoTokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples['text']) # truncation=True, padding=True

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=8, remove_columns=["text"])

In [12]:
block_size = tokenizer.model_max_length
block_size = 128

In [13]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [14]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=2000,
    num_proc=8,
)

In [15]:
lm_datasets['train']

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 110103
})

In [16]:
from transformers import AutoModelForMaskedLM
model_checkpoint_1 = 'roberta-base'
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint_1)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Aila2021/roberta-retrained-5",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=1000,
    save_total_limit=1,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets['train']
)

In [19]:
# del dataset,tokenized_datasets

In [20]:
trainer.train()

Step,Training Loss
500,1.991360
1000,1.835692
1500,1.762785
2000,1.716140
2500,1.666564
3000,1.639562


TrainOutput(global_step=3441, training_loss=1.7501533949524122)

In [21]:
trainer.save_model("/content/drive/MyDrive/Aila2021/roberta-retrained-model")